
## Installing and importing dependencies

First let us import all the modules and packages that will be required.


In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip data.shapeinstall -q tensorflow-text

In [ ]:
!pip install -q tf-models-official

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#for visualization
import seaborn as sns
import matplotlib.pyplot as plt

## Loading and Exploring dataset



There is one file provided to us in this dataset:

 * `IMDB-Dataset.csv`: The CSV file containing all the reviews and their polarity.
 
Let's read the file.


In [ ]:
import pandas as pd
data_imdb =pd.read_csv(r'../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

Thus there are 50000 rows and 2 columns in the `data` dataframe 

In [ ]:
data_imdb.head(10)

As we can see the `sentiment` column has 2 values:

   * Negative Sentiment
   * Postive Sentiment
  

Now we encode the positive sentiment to 1 and negative sentiment to 0

In [ ]:
labeling = {
    'positive':1, 
    'negative':0
}

data_imdb['sentiment'] = data_imdb['sentiment'].apply(lambda x : labeling[x])
# Output first ten rows
data_imdb.head(10)



Thus, there are no missing values in any of the columns of the dataset.

In [ ]:
label=data_imdb['sentiment']
data_imdb=data_imdb.drop(['sentiment'],axis=1)
label=label.tolist()
print(type(label))

In [ ]:
data_imdb=data_imdb['review'].tolist()
print(type(data_imdb))


In [ ]:
data=data_imdb
labels=label

In [ ]:
MAX_SEQUENCE_LENGTH = 500 # 句子 上限200个词
EMBEDDING_DIM = 10
# https://keras-cn-docs.readthedocs.io/zh_CN/latest/blog/word_embedding/
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# tokenizer
texts = data
tokenizer = Tokenizer(char_level=True) # 字向量
tokenizer.fit_on_texts(texts)
word_index = tokenizer.word_index

# sequences
sequences = tokenizer.texts_to_sequences(data)

# padding
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Found %s unique tokens.' % len(word_index))
print('Shape of data tensor:', data.shape)

In [ ]:
import random

# 打乱顺序
index = [i for i in range(len(data))]
random.shuffle(index)
data = np.array(data)[index]
labels = np.array(labels)[index]

TRAIN_SPLIT = 0.8 # 20% 测试集
TRAIN_SIZE = int(len(data) * TRAIN_SPLIT)

X_train, X_test = data[0:TRAIN_SIZE], data[TRAIN_SIZE:]
Y_train, Y_test = labels[0:TRAIN_SIZE], labels[TRAIN_SIZE:]

CNN+BILSTM



In [ ]:

from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers import Embedding
from keras.layers import Bidirectional


QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=False, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()

Let's try the preprocessing model on sample text and see the output:

In [ ]:
import tensorflow as tf
from keras import backend as K

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from keras.utils import multi_gpu_model
#from evaluate import *

EPOCHS = 3
BATCH_SIZE = 64 
VALIDATION_SPLIT = 0.3 # 30% 验证集

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('model/model-cnn-blstm.h5', save_best_only=True, save_weights_only=True)
tensor_board = TensorBoard('log/tflog-cnn-blstm', write_graph=True, write_images=True)

# model = multi_gpu_model(model)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', precision, recall, f1])

model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
          validation_split=VALIDATION_SPLIT, shuffle=True, 
          callbacks=[early_stopping, model_checkpoint, tensor_board])

In [ ]:
model.evaluate(X_test, Y_test, verbose=1, batch_size=BATCH_SIZE)

CNN+BILSTM+CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers import Embedding
from keras.layers import Bidirectional

QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Flatten())

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from keras.utils import multi_gpu_model
#from evaluate import *

EPOCHS = 3
BATCH_SIZE = 64 
VALIDATION_SPLIT = 0.3 # 30% 验证集

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('model/model-cnn-blstm-cnn.h5', save_best_only=True, save_weights_only=True)
tensor_board = TensorBoard('log/tflog-cnn-blstm-cnn', write_graph=True, write_images=True)

# model = multi_gpu_model(model)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', precision, recall, f1])

model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
          validation_split=VALIDATION_SPLIT, shuffle=True, 
          callbacks=[early_stopping, model_checkpoint, tensor_board])

In [ ]:
model.evaluate(X_test, Y_test, verbose=1, batch_size=BATCH_SIZE)

In [ ]:
BILSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization
from keras.layers import Dense, LSTM
from keras.layers import Embedding
from keras.layers import Bidirectional

QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=False, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from keras.utils import multi_gpu_model
#from evaluate import *

EPOCHS = 3
BATCH_SIZE = 64 
VALIDATION_SPLIT = 0.3 # 30% 验证集

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('model/model-blstm.h5', save_best_only=True, save_weights_only=True)
tensor_board = TensorBoard('log/tflog-blstm', write_graph=True, write_images=True)

# model = multi_gpu_model(model)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', precision, recall, f1])

model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
          validation_split=VALIDATION_SPLIT, shuffle=True, 
          callbacks=[early_stopping, model_checkpoint, tensor_board])

In [ ]:
model.evaluate(X_test, Y_test, verbose=1, batch_size=BATCH_SIZE)

In [ ]:
BiLSTM+CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Flatten
from keras.layers import Dense, LSTM, Convolution1D, MaxPooling1D
from keras.layers import Embedding
from keras.layers import Bidirectional

QA_EMBED_SIZE = 64
DROPOUT_RATE = 0.3

model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=DROPOUT_RATE)))
model.add(Convolution1D(filters=128, kernel_size=3, padding='valid', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling1D(4))
model.add(Flatten())

model.add(Dense(QA_EMBED_SIZE))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation("sigmoid"))

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
# from keras.utils import multi_gpu_model
#from evaluate import *

EPOCHS = 3
BATCH_SIZE = 64 
VALIDATION_SPLIT = 0.3 # 30% 验证集

early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('model/model-blstm-cnn.h5', save_best_only=True, save_weights_only=True)
tensor_board = TensorBoard('log/tflog-blstm-cnn', write_graph=True, write_images=True)

# model = multi_gpu_model(model)

model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', precision, recall, f1])

model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, 
          validation_split=VALIDATION_SPLIT, shuffle=True, 
          callbacks=[early_stopping, model_checkpoint, tensor_board])

In [ ]:
model.evaluate(X_test, Y_test, verbose=1, batch_size=BATCH_SIZE)